In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
train.head()

In [ ]:
train_cp = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')


In [ ]:
train.drop(columns=['Id'], axis=1, inplace=True)
train.describe()

In [ ]:
train.info()

In [ ]:
'''
81 cols -> 34 nums
now woking with missing value
'''

In [ ]:
df = pd.DataFrame({"missing":train.isnull().sum()})
df = df.sort_values(by='missing', ascending=False)
df[:20]

* PoolQc: Giá trị bị thiếu có thể do ko có hồ bơi. Nhìn qua bên cạnh chúng ta thấy có PoolArea cho ta biết diện tích hồ bơi. 1 feauture quý giá cho tra biết có hồ bơi hay ko. Kiểm tra giả thiết đúng. Thêm None vào giá trị thiếu.missing value đã đc giải quyết.
* MiscFeature: tính năng bổ sung. rất có thể là ko có. xem muc MiscVal. correct. add None to missing value
* Alley: type of alley, may be Street helpful. Replace NaN with Pave
* Fence: chất lượng hàng rào. Có thể là ko có hàng rào. Xem POOLQC just create new None row
* FireplaceQu : xem trên Fireplaces chúng ta thấy khi nó bằng 0 thì giá trị FireplaceQu bị thiếu. Thêm trường None vào giá trị bị thiếu.
* GarageYrBlt  GarageType  GarageCond  GarageFinish  GarageQual: có số dòng thiếu bằng nhau và tên gọi liên quan tới nhau, có thể ko có gara và nó liên quan tới nhau. Thêm None or 0. 
* BsmtExposure và BsmtFinType2  BsmtCond BsmtQual  BsmtFinType1 : có thể ko có ,giá trị thiếu gần nhau. 1 gias tri bi nhap sai
* MasVnrArea  MasVnrType: thieu cung nhau. xem xet 0 None
* Electrical chi co 1 gia tri: xoa hoac thay bang gia tri trung binh
* LotFrontage: khoang cach duong pho toi nha'. su li

In [ ]:
train[train['PoolQC'].isnull()][['PoolQC', 'PoolArea']]

In [ ]:
train['PoolQC'] = train['PoolQC'].fillna('None')

In [ ]:
train['PoolQC'].value_counts()

In [ ]:
train[train['MiscFeature'].isnull()][['MiscFeature', 'MiscVal']]

In [ ]:
train['MiscFeature'] = train['MiscFeature'].fillna('None')

In [ ]:
train['Alley'].unique()

In [ ]:
train[train['Alley']=='Pave'][['Alley', 'Street']]

In [ ]:
train['Alley'] = train['Alley'].fillna('Pave')

In [ ]:
train[train['Fence'].isnull()][['Fence', 'PoolQC']]

In [ ]:
train['Fence']=train['Fence'].fillna('None')

In [ ]:
train[train['FireplaceQu'].isnull()][['FireplaceQu', 'Fireplaces']]

In [ ]:
train['FireplaceQu']=train['FireplaceQu'].fillna('None')

In [ ]:
gara = ['GarageYrBlt', 'GarageType','GarageCond','GarageFinish', 'GarageQual']
train[train['GarageYrBlt'].isnull()][gara]


In [ ]:
for i in gara:
    print(train[i].unique())
'''add new colmuns None and them yearbuild can classication by era'''

In [ ]:
for i in gara:
    train[i] = train[i].fillna('None')

In [ ]:
bstm= ['BsmtExposure',  'BsmtFinType2' , 'BsmtCond', 'BsmtQual',  'BsmtFinType1','TotalBsmtSF']
train[train['BsmtExposure'].isnull()][bstm]


In [ ]:
train['BsmtExposure'][948]= 'No'


In [ ]:
for i in bstm:
    train[i] = train[i].fillna('None')

In [ ]:
mas = ['MasVnrArea',  'MasVnrType']
train[train['MasVnrArea'].isnull()][mas]


In [ ]:
train['MasVnrArea'] = train['MasVnrArea'].fillna(0)
train['MasVnrType'] = train_cp['MasVnrType'].fillna('None')

In [ ]:
train['MasVnrType'].unique()

In [ ]:
train['Electrical'] = train['Electrical'].fillna('SBrkr')

In [ ]:
np.log(train['LotFrontage']).hist()

In [ ]:
np.log(train['LotArea']).hist()

In [ ]:
train_c = train.copy()
train_c['1stFlrSF'] = (train_c['1stFlrSF']+train_c['1stFlrSF'].min())**2

train_c.corr().sort_values(by='LotFrontage')


In [ ]:
x_train = pd.concat((np.log(train['LotArea']), np.sqrt(train['LotArea']), (train_c['1stFlrSF']+train_c['1stFlrSF'].min())**2),axis=1)
y_train = train['LotFrontage']

In [ ]:
index = train[train['LotFrontage'].isnull()].index
x1 = []
y1 = []
x2= []
y2 = []
for i in train.index:
    if i in index:
        x2.append([np.log(train['LotArea'][i]), np.sqrt(train['LotArea'][i]), (train_c['1stFlrSF'][i]+train_c['1stFlrSF'].min())**2])
    else:
        x1.append([np.log(train['LotArea'][i]), np.sqrt(train['LotArea'][i]), (train_c['1stFlrSF'][i]+train_c['1stFlrSF'].min())**2])
        y1.append(y_train[i])

        

In [ ]:
x_train = np.array(x1)
y_train = np.array(y1)
x_test = np.array(x2)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(n_estimators=5,learning_rate=0.5).fit(x_train, y_train)
ypread = ada.predict(x_test)

In [ ]:
index = train[train['LotFrontage'].isnull()].index
x1 = 0
for i in index:
    train['LotFrontage'][i] = ypread[x1]
    x1 +=1
    
    

In [ ]:
train.to_csv("preprocessing.csv", header=None)